In [2]:
import torch
import torch.nn as nn

# Define the input tensor shape


# Create a random input tensor
input_tensor = torch.randn(16,1,32,32,32)

# Define the Conv3d layer in PyTorch
conv_layer = nn.ConvTranspose3d(1, out_channels=32, kernel_size=4, stride=2, padding=1)

# Apply the convolution operation
output_tensor = conv_layer(input_tensor)

# Print the shape of the output tensor
print(output_tensor.shape)


torch.Size([16, 32, 64, 64, 64])


In [21]:
import torch
import torch.nn.functional as F

# Define tensor a
a = torch.tensor([0.4835, 0.5407, 0.4840, 0.5245, 0.5569, 0.6447, 0.5125, 0.4295, 0.5018,
        0.6360, 0.5395, 0.5377, 0.5086, 0.5101, 0.4061, 0.5847])

# Define tensor b
b = torch.zeros(16)  # Create a tensor of zeros with dimension 16

# Convert tensor b to one-hot encoding
b_one_hot = F.one_hot(b.to(torch.int64), num_classes=2)  # Assuming 2 classes, change num_classes accordingly

# Calculate cross-entropy loss
loss = F.binary_cross_entropy(torch.sigmoid(a), b_one_hot.float())

print(loss.item())


ValueError: Using a target size (torch.Size([16, 2])) that is different to the input size (torch.Size([16])) is deprecated. Please ensure they have the same size.